# Software-Defined Networks Emulation in Mininet

Mininet is a network emulator that allows for rapid development of vast network architectures. Its limits lie with the hardware limits of the platform that runs the mininet. In this course, we will discuss some basic usage of mininet to emulate various network topologies.

**IMPORTANT:** Mininet's *mn* wrapper works as a CLI so we cannot use it in the Jupyter. Instead, please use the remote console to the servers having the mininet package installed and ready. These servers are at:

You can log in using standard ssh connection. From the terminal in your computer please use the following command.

```bash
ssh student@10.100.0.XX
ssh student@158.196.244.134 -p XX22  

XX: 41-53

sudo mn -c
```
The password for the connection is **student**.



## Running Mininet

To define a custom topology, you can use a sceleton class in the cell below.

In [ ]:
#Topology created with loops. STP should prevent loops.

from mininet.topo import Topo
from mininet.net import Mininet
from mininet.cli import CLI
from mininet.node import OVSSwitch

class CustomTopology(Topo):
    def build(self):
        # Add switches
        s1 = self.addSwitch('s1', cls=OVSSwitch)
        s2 = self.addSwitch('s2', cls=OVSSwitch)
        s3 = self.addSwitch('s3', cls=OVSSwitch)

        # Add hosts
        h1 = self.addHost('h1')
        h2 = self.addHost('h2')
        h3 = self.addHost('h3')

        # Add links with loops
        self.addLink(s1, s2)
        self.addLink(s2, s3)
        self.addLink(s3, s1)
        self.addLink(s1, h1)
        self.addLink(s2, h2)
        self.addLink(s3, h3)

def run_topology():
    topo = CustomTopology()
    net = Mininet(topo=topo, controller=None, switch=OVSSwitch)  # Set switch type and disable default controller

    # This topology has intentional loops, and the Spanning Tree Protocol (STP) is employed to prevent broadcast storms and ensure a loop-free topology.
    net.start()

    # Print network configuration
    print("Topology created with loops. STP should prevent loops.")

    # Start the CLI
    CLI(net)

    net.stop()

if __name__ == '__main__':
    run_topology()


In [ ]:
from mininet.net import Mininet
from mininet.topo import Topo
from mininet.node import OVSSwitch, Controller
from mininet.link import TCLink

class CustomTopology(Topo):
    def build(self):
        # Add switches
        s1 = self.addSwitch('s1')
        s2 = self.addSwitch('s2')

        # Add hosts
        h1 = self.addHost('h1', ip='10.0.0.1/24')
        h2 = self.addHost('h2', ip='10.0.0.2/24')
        h3 = self.addHost('h3', ip='10.0.0.3/24')
        h4 = self.addHost('h4', ip='10.0.0.4/24')

        # Add links with QoS parameters
        self.addLink(h1, s1, bw=1000, delay='1ms', loss=0, max_queue_size=1000, use_htb=True)
        self.addLink(h2, s1, bw=1000, delay='1ms', loss=0, max_queue_size=1000, use_htb=True)
        self.addLink(h3, s2, bw=1000, delay='1ms', loss=0, max_queue_size=1000, use_htb=True)
        self.addLink(h4, s2, bw=1000, delay='1ms', loss=0, max_queue_size=1000, use_htb=True)
        self.addLink(s1, s2, bw=1000, delay='5ms', loss=0, max_queue_size=1000, use_htb=True)

def start_mininet():
    topo = CustomTopology()
    net = Mininet(topo=topo, switch=OVSSwitch, link=TCLink, controller=Controller)
    net.start()

    # Set QoS parameters for hosts
    for host in net.hosts:
        host.cmd('tc qdisc add dev {}-eth0 root handle 1: htb default 11'.format(host.name))
        host.cmd('tc class add dev {}-eth0 parent 1: classid 1:1 htb rate 1000mbit'.format(host.name))
        host.cmd('tc class add dev {}-eth0 parent 1:1 classid 1:11 htb rate 1000mbit'.format(host.name))
        host.cmd('tc qdisc add dev {}-eth0 parent 1:11 handle 10: netem delay 5ms'.format(host.name))

    net.interact()

if __name__ == '__main__':
    start_mininet()


In [ ]:
from mininet.topo import Topo
from mininet.net import Mininet
from mininet.node import CPULimitedHost
from mininet.link import TCLink
from mininet.util import dumpNodeConnections

class CustomTopology(Topo):
    def build(self):
        # Add two hosts, one switch, and a controller
        h1 = self.addHost('h1')
        h2 = self.addHost('h2')
        s1 = self.addSwitch('s1')
        c0 = self.addController('c0')

        # Add links with custom bandwidth and delay
        self.addLink(h1, s1, bw=10, delay='5ms', max_queue_size=1000, use_tbf=True)
        self.addLink(h2, s1, bw=5, delay='10ms', max_queue_size=1000, use_tbf=True)

def run_custom_topology():
    topo = CustomTopology()
    net = Mininet(topo=topo, host=CPULimitedHost, link=TCLink)
    net.start()

    print("Dumping host connections")
    dumpNodeConnections(net.hosts)

    # Set DSCP code for traffic from h1 to h2
    h1, h2 = net.get('h1', 'h2')
    h1.cmd('tc qdisc add dev h1-eth0 root handle 1: htb default 1')
    h1.cmd('tc class add dev h1-eth0 parent 1: classid 1:1 htb rate 10mbit')
    h1.cmd('tc filter add dev h1-eth0 parent 1: protocol ip prio 1 u32 match ip dscp 10 0xff flowid 1:1')

    # Start a simple HTTP server on h2
    h2.cmd('python -m SimpleHTTPServer 80 &')

    # Test connectivity
    net.pingAll()

    # Stop the network
    net.stop()

if __name__ == '__main__':
    run_custom_topology()


### Task

* Add network loss to the topology as well.
* Measure the network performance using iperf tool.